In [26]:
import pandas as pd
import numpy as np
import h5py

def create_interaction_analysis_dataframe(hdf5_path, target_frames=None):
    """
    Create a merged DataFrame specifically designed for Dominoes analysis
    
    Selection Principles:
    1. Core interaction data: position, velocity - for distance calculation and interaction detection
    2. Object identification: ID, type - for result interpretation
    3. Physical properties: mass, size - affecting interaction thresholds and intensity
    4. Task-related: target object markers - focus on specific objects
    5. Time information: frame number - for temporal analysis
    
    Exclusion Principles:
    1. Static unchanged data: color, initial position etc. - no impact on dynamic interactions
    2. Visual related: images, segmentation masks - irrelevant to physical interactions
    3. Experimental control: seed, version etc. - irrelevant to interaction analysis
    4. Redundant data: direction vectors (derivable from position) - avoid data redundancy
    """
    
    with h5py.File(hdf5_path, 'r') as f:
        # Get static data
        static_data = {}
        static_keys = ['object_ids', 'model_names', 'scale', 'initial_position',
                      'target_id', 'probe_id',]
        
        for key in static_keys:
            if key in f['/static']:
                static_data[key] = f[f'/static/{key}'][()]
        
        # Determine frames to analyze
        total_frames = len(f['/frames'].keys())
        if target_frames is None:
            target_frames = range(min(100, total_frames))  # Analyze first 100 frames by default
        
        # Store data from all frames
        all_frame_data = []
        
        for frame_idx in target_frames:
            frame_key = f"{frame_idx:04d}"
            if frame_key not in f['/frames']:
                continue
                
            frame_group = f[f'/frames/{frame_key}']
            
            # Get dynamic data for each frame
            frame_data = {
                'frame': frame_idx,
                'positions': frame_group['objects/positions'][()],
                'velocities': frame_group['objects/velocities'][()],
                'angular_velocities': frame_group['objects/angular_velocities'][()],
                'rotations': frame_group['objects/rotations'][()],
            }
            
            all_frame_data.append(frame_data)
    
    # Create flattened DataFrame
    df_rows = []
    
    for frame_data in all_frame_data:
        frame_num = frame_data['frame']
        num_objects = len(static_data['object_ids'])
        
        for obj_idx in range(num_objects):
            # Basic object information
            row = {
                'frame': frame_num,
                'object_id': static_data['object_ids'][obj_idx],
                'model_name': static_data['model_names'][obj_idx].decode() if isinstance(static_data['model_names'][obj_idx], bytes) else static_data['model_names'][obj_idx],
                
                # Position and motion information - core interaction data
                'position(x,y,z)': frame_data['positions'][obj_idx],
                'velocity(x,y,z)': frame_data['velocities'][obj_idx],
                'angular_velocity(x,y,z)': frame_data['angular_velocities'][obj_idx],
                
                # Physical properties - 3D scaling factors for interaction boundary calculation
                'scale(x,y,z)': static_data['scale'][obj_idx],
                
                # Computed properties
                'speed': np.linalg.norm(frame_data['velocities'][obj_idx]),  # Total speed
                'angular_speed': np.linalg.norm(frame_data['angular_velocities'][obj_idx]),  # Total angular speed
                
                # Identification properties
                'is_target': static_data['object_ids'][obj_idx] == static_data['target_id'],
                'is_probe': static_data['object_ids'][obj_idx] == static_data['probe_id'],
            }
            
            df_rows.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(df_rows)
    
    # Add computed columns: velocity changes relative to previous frame
    df = df.sort_values(['object_id', 'frame']).reset_index(drop=True)
    
    for obj_id in df['object_id'].unique():
        obj_mask = df['object_id'] == obj_id
        obj_data = df[obj_mask].copy()
        
        # Calculate velocity changes
        obj_data['vel_change_magnitude'] = 0.0
        obj_data['pos_change_magnitude'] = 0.0
        
        for i in range(1, len(obj_data)):
            prev_vel = obj_data.iloc[i-1]['velocity(x,y,z)']
            curr_vel = obj_data.iloc[i]['velocity(x,y,z)']
            vel_change = np.linalg.norm(np.array(curr_vel) - np.array(prev_vel))
            obj_data.iloc[i, obj_data.columns.get_loc('vel_change_magnitude')] = vel_change
            
            prev_pos = obj_data.iloc[i-1]['position(x,y,z)']
            curr_pos = obj_data.iloc[i]['position(x,y,z)']
            pos_change = np.linalg.norm(np.array(curr_pos) - np.array(prev_pos))
            obj_data.iloc[i, obj_data.columns.get_loc('pos_change_magnitude')] = pos_change
        
        # Update original DataFrame
        df.loc[obj_mask, 'vel_change_magnitude'] = obj_data['vel_change_magnitude'].values
        df.loc[obj_mask, 'pos_change_magnitude'] = obj_data['pos_change_magnitude'].values
    
    return df

In [39]:
HDF5_FILE = "physion_train/dynamics_training/Dominoes/Dominoes/pilot_dominoes_0mid_d3chairs_o1plants_tdwroom_0013.hdf5"
df = create_interaction_analysis_dataframe(HDF5_FILE, target_frames=range(45))  # Analyze first 45 frames
df

,frame,object_id,model_name,"position(x,y,z)","velocity(x,y,z)","angular_velocity(x,y,z)","scale(x,y,z)",speed,angular_speed,is_target,is_probe,vel_change_magnitude,pos_change_magnitude
0,0,1,cube,"[0.85, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.5, 0.01, 2.0]",0.0,0.0,False,False,0.0,0.0
1,1,1,cube,"[0.85, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.5, 0.01, 2.0]",0.0,0.0,False,False,0.0,0.0
2,2,1,cube,"[0.85, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.5, 0.01, 2.0]",0.0,0.0,False,False,0.0,0.0
3,3,1,cube,"[0.85, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.5, 0.01, 2.0]",0.0,0.0,False,False,0.0,0.0
4,4,1,cube,"[0.85, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.5, 0.01, 2.0]",0.0,0.0,False,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,40,7,naughtone_pinch_stool_chair,"[-1.6846377, 0.0, -0.75]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.75139236, 0.75139236, 0.75139236]",0.0,0.0,False,False,0.0,0.0
311,41,7,naughtone_pinch_stool_chair,"[-1.6846377, 0.0, -0.75]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.75139236, 0.75139236, 0.75139236]",0.0,0.0,False,False,0.0,0.0
312,42,7,naughtone_pinch_stool_chair,"[-1.6846377, 0.0, -0.75]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.75139236, 0.75139236, 0.75139236]",0.0,0.0,False,False,0.0,0.0
313,43,7,naughtone_pinch_stool_chair,"[-1.6846377, 0.0, -0.75]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.75139236, 0.75139236, 0.75139236]",0.0,0.0,False,False,0.0,0.0


# Method 1
### Simple threshold-based interaction analysis

In [40]:
interaction_threshold = 0.01
actual_interactions = df[df['vel_change_magnitude'] > interaction_threshold]
print(f"Detected {actual_interactions.shape[0]} potential interaction moments")
actual_interactions.head()

Detected 15 potential interaction moments


,frame,object_id,model_name,"position(x,y,z)","velocity(x,y,z)","angular_velocity(x,y,z)","scale(x,y,z)",speed,angular_speed,is_target,is_probe,vel_change_magnitude,pos_change_magnitude
120,30,3,cube,"[-0.25011, 0.0056406558, -5.735294e-05]","[0.76923615, 0.13679124, 0.37937394]","[1.2129755, 0.0012393355, -2.4590948]","[0.1, 0.6337526, 0.20849508]",0.868539,2.741981,False,True,0.868540,0.001499
121,31,3,cube,"[-0.25018468, 0.007019222, -0.00010653073]","[0.76178664, 0.10291251, 0.3801516]","[1.2170403, -0.023935214, -2.4345744]","[0.1, 0.6337526, 0.20849508]",0.857569,2.721932,False,True,0.034697,0.001381
122,32,3,cube,"[-0.2501095, 0.008347005, -6.9005415e-05]","[0.76056516, 0.07551493, 0.38045663]","[1.1916152, 0.00016235933, -2.3820925]","[0.1, 0.6337526, 0.20849508]",0.853762,2.663515,False,True,0.027426,0.001330
123,33,3,cube,"[-0.25002918, 0.0096695125, -4.2480417e-05]","[0.75707316, 0.05276365, 0.37718496]","[1.1833792, -0.02356462, -2.3743875]","[0.1, 0.6337526, 0.20849508]",0.847474,2.653047,False,True,0.023249,0.001325
124,34,3,cube,"[-0.24988851, 0.010975212, 2.7690083e-05]","[0.7560505, 0.029785037, 0.37563726]","[1.1718189, -0.00046624802, -2.358696]","[0.1, 0.6337526, 0.20849508]",0.844750,2.633744,False,True,0.023053,0.001315


In [41]:
target_movement = df[df['is_target'] == True]
target_movement.head()

,frame,object_id,model_name,"position(x,y,z)","velocity(x,y,z)","angular_velocity(x,y,z)","scale(x,y,z)",speed,angular_speed,is_target,is_probe,vel_change_magnitude,pos_change_magnitude
45,0,2,cube,"[0.25000012, 0.003317535, -5.5804534e-09]","[-0.005663872, -0.00096836686, -0.0019756875]","[-0.009051699, 0.0013341487, 0.021036068]","[0.1, 0.5, 0.25]",0.006076,0.022940,True,False,0.000000,0.000000
46,1,2,cube,"[0.24999794, 0.003975123, 1.8131122e-06]","[-0.006887803, -0.0011759028, -0.0006539526]","[-0.0041053304, 0.0011296391, 0.025976282]","[0.1, 0.5, 0.25]",0.007018,0.026323,True,False,0.001813,0.000658
47,2,2,cube,"[0.24999692, 0.0041023493, 3.2868566e-06]","[-0.0027835974, -0.00047539175, -0.00014235772]","[-0.0011786984, 0.0004250225, 0.010525421]","[0.1, 0.5, 0.25]",0.002827,0.010600,True,False,0.004195,0.000127
48,3,2,cube,"[0.24999765, 0.0041294396, 2.8034233e-06]","[-1.8655788e-05, -3.1515956e-06, 3.4449273e-05]","[0.00013203628, -7.82634e-06, 7.854216e-05]","[0.1, 0.5, 0.25]",0.000039,0.000154,True,False,0.002811,0.000027
49,4,2,cube,"[0.24999866, 0.0041380823, 1.7615487e-06]","[0.0010437903, 0.00017837435, 7.907035e-05]","[0.0005437709, -0.00016773885, -0.003941009]","[0.1, 0.5, 0.25]",0.001062,0.003982,True,False,0.001079,0.000009


# Method 2 
### Distance-based interaction analysis

In [42]:
def add_pairwise_distances(df, distance_threshold=2.0):
    """
    Add pairwise distance information for candidate interaction pair detection
    """
    distance_data = []
    
    for frame in df['frame'].unique():
        frame_data = df[df['frame'] == frame]
        objects = frame_data[['object_id', 'position(x,y,z)', 'scale(x,y,z)']].values
        
        for i in range(len(objects)):
            for j in range(i+1, len(objects)):
                obj1_id, position1, scale1 = objects[i]
                obj2_id, position2, scale2 = objects[j]
                
                # Calculate center-to-center distance
                distance = np.linalg.norm(np.array(position2) - np.array(position1))
                
                # ***Estimate contact threshold (based on object sizes)***
                contact_threshold = (min(scale1) + min(scale2)) / 2
                
                # Only record close-proximity objects that may interact
                if distance <= distance_threshold:
                    distance_data.append({
                        'frame': frame,
                        'object1_id': int(obj1_id),
                        'object2_id': int(obj2_id),
                        'distance': distance,
                        'contact_threshold': contact_threshold,
                        'is_potential_interaction': distance <= contact_threshold,
                        'distance_ratio': distance / contact_threshold  # <1 indicates potential contact
                    })
    
    distance_df = pd.DataFrame(distance_data)
    return distance_df

In [43]:
def analyze_interactions(hdf5_path):
    """Complete interaction analysis workflow"""
    # Create distance analysis DataFrame
    print("\nCalculating pairwise distances...")
    distance_df = add_pairwise_distances(df, distance_threshold=1.5)
    
    print(f"Distance DataFrame created: {distance_df.shape}")
    print(f"Detected {distance_df['is_potential_interaction'].sum()} potential interaction moments")
    
    return distance_df

In [44]:
distance_df = analyze_interactions(HDF5_FILE)
distance_df


Calculating pairwise distances...
Distance DataFrame created: (450, 7)
Detected 180 potential interaction moments


,frame,object1_id,object2_id,distance,contact_threshold,is_potential_interaction,distance_ratio
0,0,1,2,0.600009,0.055000,False,10.909256
1,0,1,3,1.100005,0.055000,False,20.000094
2,0,1,4,1.019176,1.258383,True,0.809909
3,0,2,3,0.500000,0.100000,False,5.000002
4,0,2,4,1.278616,1.303383,True,0.980998
...,...,...,...,...,...,...,...
445,44,2,5,1.142252,1.057593,False,1.080048
446,44,2,6,1.197101,0.550000,False,2.176548
447,44,3,5,1.000208,1.057593,True,0.945740
448,44,3,6,1.038646,0.550000,False,1.888446


In [45]:
candidates = distance_df[distance_df['is_potential_interaction'] == True]
candidates.head()

,frame,object1_id,object2_id,distance,contact_threshold,is_potential_interaction,distance_ratio
2,0,1,4,1.019176,1.258383,True,0.809909
4,0,2,4,1.278616,1.303383,True,0.980998
7,0,3,5,1.001357,1.057593,True,0.946827
9,0,5,6,0.060947,1.507593,True,0.040427
12,1,1,4,1.019176,1.258383,True,0.809909


# Method 3

In [46]:
import numpy as np
import pandas as pd

def add_pairwise_distances_with_velocity_analysis(df, distance_threshold=2.0, velocity_change_threshold=0.01):
    """
    Add pairwise distance information and velocity-based interaction detection
    """
    distance_data = []
    
    for frame in df['frame'].unique():
        frame_data = df[df['frame'] == frame]
        objects = frame_data[['object_id', 'position(x,y,z)', 'scale(x,y,z)', 'velocity(x,y,z)', 'vel_change_magnitude']].values
        
        for i in range(len(objects)):
            for j in range(i+1, len(objects)):
                obj1_id, position1, scale1, velocity1, vel_change1 = objects[i]
                obj2_id, position2, scale2, velocity2, vel_change2 = objects[j]
                
                # Calculate distance
                distance = np.linalg.norm(np.array(position2) - np.array(position1))
                
                # Contact threshold based on object sizes
                contact_threshold = (min(scale1) + min(scale2)) / 2
                
                if distance <= distance_threshold:
                    # Velocity analysis
                    has_velocity_change = (vel_change1 > velocity_change_threshold or 
                                         vel_change2 > velocity_change_threshold)
                    relative_velocity = np.linalg.norm(np.array(velocity2) - np.array(velocity1))
                    
                    # Interaction confidence
                    confidence = 0
                    evidence = []
                    
                    if distance <= contact_threshold:
                        confidence += 40
                        evidence.append("Contact")
                    if has_velocity_change:
                        confidence += 30
                        evidence.append("Velocity change")
                    if relative_velocity > 0.01:
                        confidence += 20
                        evidence.append("Relative motion")
                    
                    is_interaction = confidence >= 60 and has_velocity_change
                    
                    distance_data.append({
                        'frame': frame,
                        'object1_id': int(obj1_id),
                        'object2_id': int(obj2_id),
                        'distance': distance,
                        'contact_threshold': contact_threshold,
                        'is_potential_interaction': distance <= contact_threshold,
                        'distance_ratio': distance / contact_threshold,
                        'obj1_vel_change': vel_change1,
                        'obj2_vel_change': vel_change2,
                        'has_velocity_change': has_velocity_change,
                        'relative_velocity': relative_velocity,
                        'interaction_confidence': min(confidence, 100),
                        'is_actual_interaction': is_interaction,
                        'interaction_evidence': "; ".join(evidence) if evidence else "No evidence"
                    })
    
    return pd.DataFrame(distance_data)

def analyze_interactions_enhanced(df):
    """
    Enhanced interaction detection analysis
    """
    print("Starting enhanced interaction analysis...")
    
    distance_df = add_pairwise_distances_with_velocity_analysis(df, distance_threshold=2.0, velocity_change_threshold=0.01)
    
    potential_interactions = distance_df[distance_df['is_potential_interaction'] == True]
    actual_interactions = distance_df[distance_df['is_actual_interaction'] == True]
    
    print(f"Analysis complete: {len(distance_df)} interaction records")
    print(f"Potential interactions (distance): {len(potential_interactions)}")
    print(f"Actual interactions (comprehensive): {len(actual_interactions)}")
    
    if len(actual_interactions) > 0:
        print("\nDetected actual interactions:")
        print(actual_interactions[['frame', 'object1_id', 'object2_id', 'interaction_confidence', 'interaction_evidence']].to_string())
    
    return distance_df

In [47]:
analyze_interactions_enhanced_df = analyze_interactions_enhanced(df)

Starting enhanced interaction analysis...
Analysis complete: 630 interaction records
Potential interactions (distance): 180
Actual interactions (comprehensive): 15

Detected actual interactions:
     frame  object1_id  object2_id  interaction_confidence                       interaction_evidence
430     30           3           5                      90  Contact; Velocity change; Relative motion
444     31           3           5                      90  Contact; Velocity change; Relative motion
458     32           3           5                      90  Contact; Velocity change; Relative motion
472     33           3           5                      90  Contact; Velocity change; Relative motion
486     34           3           5                      90  Contact; Velocity change; Relative motion
500     35           3           5                      90  Contact; Velocity change; Relative motion
514     36           3           5                      90  Contact; Velocity change; Relat